In [1]:
# Imports
import networkx as nx
import numpy as np
import cvxpy as cp
import requests
import pandas as pd
import osmnx as ox

# Helper Functions

def calculateWeight(data):
    inf = 1e6

    weight = (1 - data['speedOrMaxSpeed']) * getInverse(data['maxSpeed']) * data['length'] + data['speedOrMaxSpeed'] * getInverse(data['speed']) * data['length'] + inf * data['noWay'] + inf * data['isClosed']
    
    # if data['speedOrMaxSpeed'] == 1:
    #     weight = getInverse(data['speed']) * data['length'] + inf * data['noWay'] + inf * data['isClosed']
    # else:
    #     weight = getInverse(data['maxSpeed']) * data['length'] + inf * data['noWay'] + inf * data['isClosed']
        
    return weight

def updateEdgeWeight(graph, source, target, data):
    graph[source][target][0]['weight'] = calculateWeight(data)

def updateGraphWeights(graph):
    for (i, j, data) in graph.edges(data=True):
        updateEdgeWeight(graph, i, j, data)
    
def getPathWeight(path, graph):
    weights = 0
    for i in range(len(path) - 1):
        j = i + 1
        source = path[i]
        target = path[j]
        weights += graph[source][target][0]['weight']
        
    return weights
        
def getInverse(speed):
    return 1/speed

def addReverseEdges(graph):
    for (i, j) in graph.edges():
        if (j, i) not in graph.edges():
            graph.add_edge(
                            j, i, 
                            weight = np.nan,
                            noWay = 1,
                            isClosed = 0,
                            length = 0,
                            speed = graph[i][j][0]['speed'],
                            maxSpeed = graph[i][j][0]['maxSpeed'],
                            speedOrMaxSpeed = 1
                        )
            updateEdgeWeight(graph, j, i, graph[j][i][0])
    
def cleanGraphAttributes(graph):
    for (i, j) in graph.edges():
        if graph[i][j][0]['isClosed'] == 1:
            graph[i][j][0]['length'] = 0
    
def prepareGraph(graph):
    cleanGraphAttributes(G)
    updateGraphWeights(G)
    addReverseEdges(graph)
    
    
def inverseShortestPath(graph, desiredPath):
    # Constants
    inf = 1e6
    epsilon = 1e-6
    possibleMaxSpeeds = [10, 20, 30, 40, 50, 60, 70, 80, 90]
    inversePossibleMaxSpeeds = [getInverse(s) for s in possibleMaxSpeeds] + [0]
    inversePossibleMaxSpeeds = np.asarray(inversePossibleMaxSpeeds)
    
    # Some graph and path data
    n = len(graph.nodes())
    m = len(graph.edges())
    source = desiredPath[0]
    target = desiredPath[len(desiredPath) - 1]

    # Original Graph data
    noWay = []
    areClosed = []
    inverseSpeeds = []
    inverseMaxSpeeds = []
    lengths = []
    maxSpeeds_H1E = []
    speedOrMaxSpeed_original = []
    
    # Edges data, and their indecies
    edges = []
    edgeIndex = {}
    for (i, j, data) in graph.edges(data=True):
        # Edges Index
        edgeIndex[i,j] = len(edges)
        # Add edge
        edges.append([i, j])
        
        # Data:
        noWay.append(data['noWay'])
        areClosed.append(data['isClosed'])
        inverseSpeeds.append(getInverse(data['speed']))
        inverseMaxSpeeds.append(getInverse(data['maxSpeed']))
        lengths.append(data['length'])
        speedOrMaxSpeed_original.append(data['speedOrMaxSpeed'])
        
        # hot 1 encoding original data
        hot1E = []
        for ms in inversePossibleMaxSpeeds:
            if getInverse(data['maxSpeed']) == ms:
                hot1E.append(1)
            else:
                hot1E.append(0)
        maxSpeeds_H1E.append(hot1E)
        
        
        
    # Nodes data, and their indecies
    nodeIndex = {}
    nodes = []
    for n in graph.nodes:
        # Node Index
        nodeIndex[n] = len(nodes)
        # Add Node
        nodes.append(n)
        
        
    # Ax = b
    A = np.zeros([len(nodes), len(edges)])
    b = np.zeros(len(nodes))

    for i in range(len(nodes)):
        for neighbour in graph.adj[nodes[i]]:
            # Filling A
            j = edgeIndex[nodes[i], neighbour]
            A[i,j] = 1
            if (neighbour, nodes[i]) in edgeIndex:
                j = edgeIndex[neighbour, nodes[i]]
                A[i,j] =-1
            
        # Filling b
        if nodes[i] == source:
            b[i] = 1
        if nodes[i] == target:
            b[i] = -1
            
            
    # optimal x
    path = nx.shortest_path(graph, source=desiredPath[0], target=desiredPath[-1], weight="weight")
    xstar = np.zeros(len(edges))
    for p in range(len(path)-1):
        j = edgeIndex[path[p], path[p+1]]
        xstar[j] = 1

    # desired x
    xzero = np.zeros(len(edges))
    for p in range(len(desiredPath)-1):
        j = edgeIndex[desiredPath[p], desiredPath[p+1]]
        xzero[j] = 1
        


    # LP (ISP):
    
    # Original Data as numpy array. (for mathematical applications)
    noWay_original = np.asarray(noWay)
    areClosed_original = np.asarray(areClosed)
    inverseSpeeds_original = np.asarray(inverseSpeeds)
    inverseMaxSpeeds_original = np.asarray(inverseMaxSpeeds)
    maxSpeeds_H1E_original = np.asarray(maxSpeeds_H1E)
    speedOrMaxSpeed_original = np.asarray(speedOrMaxSpeed_original)
    
    
    # Variables
    pi_ = cp.Variable(len(nodes)) #(2d)
    lambda_ = cp.Variable(len(edges)) #(2d)
    
    noWay_ = cp.Variable(len(edges), boolean=True)
    areClosed_ = cp.Variable(len(edges), boolean=True)
    inverseSpeeds_ = cp.Variable(len(inverseSpeeds_original))
    inverseMaxSpeeds_ = cp.Variable(len(edges))
    maxSpeeds_H1E_ = cp.Variable(maxSpeeds_H1E_original.shape, boolean=True)
    # If 1, then change speed, else if 0 change maxSpeed.
    speedOrMaxSpeed_ = cp.Variable(len(edges), boolean=True)
        
        
    # Constraints
    constraints = []
            
            
    # Hot 1 Encoding    
    for row in maxSpeeds_H1E_:
        constraints.append( sum(row) == 1)
        
    
    for j in range(len(edges)):
        
        # Hot 1 Encoding, For all edges in G
        constraints.append( inverseMaxSpeeds_[j] == inversePossibleMaxSpeeds.T @ maxSpeeds_H1E_[j] )
        
        
        if xzero[j] == 1: #for all j in desired path
            d_j = inverseSpeeds_[j] * lengths[j] + inverseMaxSpeeds_[j] * lengths[j] + inf * noWay_[j] + inf * areClosed_[j]
        
            # sum_i a_ij * pi_i = d_j,               (2b)
            constraints.append( cp.sum(cp.multiply(A[:,j], pi_)) == d_j )
            
            
            # if speed/maxSpeed >= 3/4 choose the maxSpeed to change, else the speed.
            if inverseMaxSpeeds_original[j] / inverseSpeeds_original[j] >= 3/4:
                # Change maxSpeed
                constraints.append( speedOrMaxSpeed_[j] == 0 )
                constraints.append( inverseMaxSpeeds_[j] >= epsilon )
            else:
                # Change speed
                constraints.append( speedOrMaxSpeed_[j] == 1 )
                
                
            # Lower bound and Upper bounds are 0, if we use the other metric. For speed and max Speed.
            constraints.append( (1 - speedOrMaxSpeed_[j]) * min(inversePossibleMaxSpeeds) <= inverseMaxSpeeds_[j] )
            constraints.append( (1 - speedOrMaxSpeed_[j]) * max(inversePossibleMaxSpeeds) >= inverseMaxSpeeds_[j] )
            
            constraints.append( speedOrMaxSpeed_[j] * inverseMaxSpeeds_original[j] <= inverseSpeeds_[j] )
            constraints.append( speedOrMaxSpeed_[j] * inverseSpeeds_original[j] >= inverseSpeeds_[j] )
            
        else: # for all j not in desired path
            d_j = inverseSpeeds_[j] * lengths[j] + inf * noWay_[j] + inf * areClosed_[j]
            
            # sum_i a_ij * pi_i + lambda_j = d_j,    (2c)
            constraints.append( cp.sum(cp.multiply(A[:,j], pi_)) + lambda_[j] == d_j )
            
            # Do not change datta
            constraints.append( noWay_[j] == noWay_original[j] )
            constraints.append( areClosed_[j] == areClosed_original[j] )
            constraints.append( inverseSpeeds_[j] == inverseSpeeds_original[j] )
            constraints.append( inverseMaxSpeeds_[j] == inverseMaxSpeeds_original[j] )
            constraints.append( maxSpeeds_H1E_[j] == maxSpeeds_H1E_original[j] )
            constraints.append( speedOrMaxSpeed_[j] == speedOrMaxSpeed_original[j] )
            
            # # Keep using speed, and not maxSpeed
            # constraints.append( speedOrMaxSpeed_[j] == 1)
            
            # (2e)
            constraints.append( lambda_[j] >= 0 )
    
    penalty1 = 1
    penalty2 = 100
    # Cost function, split up
    cost1 = cp.norm1(cp.multiply(inverseSpeeds_ - inverseSpeeds_original, penalty1))
    cost2 = cp.norm1(cp.multiply(inverseMaxSpeeds_ - inverseMaxSpeeds_original, penalty2))
    cost3 = cp.norm1(noWay_ - noWay_original)
    cost4 = cp.norm1(areClosed_ - areClosed_original)
            
            
    # Final Cost funnction
    cost = cost1 + cost2 + cost3 + cost4
    
        
    # Forming the problem
    prob = cp.Problem(cp.Minimize(cost), constraints)
    
    # Solve the problem
    #prob.solve(solver=cp.GUROBI, verbose=True) #Detailed
    prob.solve(solver=cp.GUROBI) # using gurobi
    print("\nThe optimal value is", prob.value)
    
    #Helper print statements
    print('original speedInv: ', inverseSpeeds_original)
    print('optimal speedInv: ', inverseSpeeds_.value)
    print('\n')
    print('original speedMaxInv: ', inverseMaxSpeeds_original)
    print('optimal speedMaxInv: ', inverseMaxSpeeds_.value)
    # print('\n')
    # print('original noWay: ', noWay_original)
    # print('optimal noWay: ', noWay_.value)
    # print('\n')
    # print('original areClosed: ', areClosed_original)
    # print('optimal areClosed: ', areClosed_.value)
    # print('\n')
    # print('original speedOrMaxSpeed: ', speedOrMaxSpeed_original)
    # print('optimal speedOrMaxSpeed: ', speedOrMaxSpeed_.value)
    
    
    # Creating the new Graph
    newGraph = nx.MultiDiGraph()
    
    for (i, j), index in edgeIndex.items():
        if speedOrMaxSpeed_.value[index] == 0:
            s = getInverse(inverseSpeeds_original[index])
        else:
            s = getInverse(inverseSpeeds_.value[index])
            
        if speedOrMaxSpeed_.value[index] == 1:
            ms = getInverse(inverseMaxSpeeds_original[index])
        else:
            ms = getInverse(inverseMaxSpeeds_.value[index])
            
        newGraph.add_edge(
                            i, j,
                            weight = np.nan, 
                            noWay = noWay_.value[index], 
                            isClosed = areClosed_.value[index], 
                            length = graph[i][j][0]['length'], 
                            speed = s,
                            maxSpeed = ms,
                            speedOrMaxSpeed = speedOrMaxSpeed_.value[index]
                        )
        
    updateGraphWeights(newGraph)
        
        
    # Final checks
    sp = nx.shortest_path(newGraph, source=desiredPath[0], target=desiredPath[-1], weight="weight")
    
    desiredPathWeight = getPathWeight(desiredPath, newGraph)
    optimalPathWeight = getPathWeight(sp, newGraph)
    print('\n')
    
    if (desiredPathWeight - optimalPathWeight) <= epsilon:
        print('The desired Path is equal to the Shortest Path')
    elif desiredPathWeight < optimalPathWeight:
        print('The desired Path is better than the Shortest Path')
    elif desiredPathWeight > optimalPathWeight:
        print('The desired Path is worse than the Shortest Path')
    
    
    print('Optimal Path Weight = ', optimalPathWeight)
    print('The path is: ', sp)
    print('numbers after decimal point: ', len(str(optimalPathWeight).replace('.','')) - 1)
    print('\n')
    print('Desired Path Weight = ', desiredPathWeight)
    print('The path is: ', desiredPath)
    print('numbers after decimal point: ', len(str(desiredPathWeight).replace('.','')) - 1)
        
    return newGraph

# Gets the traffic data from tomtom for all the edges
def add_full_traffic_data(edges):
    
    edges.name = (edges.name).astype(str)
    names = edges.name.unique()
    
    for name in names:
        add_specific_traffic_data(edges, name)
        


# Gets the traffic data from tomtom for a specific street (all edges with this street name)
def add_specific_traffic_data(edges, name):
    # Create request URL
    BASE_URL = "https://api.tomtom.com/"
    API_KEY = "ThfURaOhGRaty7mx9AKALwrbyIZApHd8"
    #zoom = 22
    TRAFFIC_URL = "traffic/services/4/flowSegmentData/absolute/22/json?key="
    UNIT = "unit=mph"
    ROAD_CLOSURE = "roadClosure=True"
    PARAMS = "&" + UNIT + "&" + ROAD_CLOSURE
    
    coordinates = list(edges.loc[edges['name'] == name].iloc[0].geometry.coords)[0]
    longitude = str(coordinates[0])
    latitude = str(coordinates[1])
    location = "point=" + latitude + ","+ longitude

    request_url = BASE_URL + TRAFFIC_URL + API_KEY + "&" + location + PARAMS
    # Get data
    response = requests.get(request_url)
    data = response.json()['flowSegmentData']
    traffic = pd.json_normalize(data)
    
    #edges.loc[edges['name'] == name, 'road_type'] = traffic.frc[0]
    edges.loc[edges['name'] == name, 'speed'] = traffic.currentSpeed[0]
    #edges.loc[edges['name'] == name, 'free_flow_speed'] = traffic.freeFlowSpeed[0]
    #edges.loc[edges['name'] == name, 'traffic_confidence'] = traffic.confidence[0]
    edges.loc[edges['name'] == name, 'isClosed'] = (traffic.roadClosure[0]).astype(int)


In [2]:
address = '30 Aldwych, London WC2B 4BG'
G = ox.graph_from_address(address, network_type="drive", dist=1000)

In [ ]:
def convertMaxSpeedToInt(speed):
    if type(speed) == list:
        speeds = []
        for i in speed:
            speeds.append(convertMaxSpeedToInt(i))
        
        return max(speeds)
    else:
        return int((speed.split(" ")[0]))
    

def fixGraphData(graph):
    nodes, edges = ox.graph_to_gdfs(graph)
    
    edges = edges[['osmid', 'name', 'maxspeed', 'length', 'geometry', 'oneway']]
    
    edges.rename(columns={'maxspeed': 'maxSpeed'}, inplace=True)
    
    # Converting length from meters to miles
    meterInMiles = 0.00062137
    edges['length'] = edges['length'].apply(lambda x: x * meterInMiles)
    
    # Fillna with mode, and convert string to int
    edges['maxSpeed'].fillna((edges['maxSpeed'].mode()[0]), inplace=True)
    edges['maxSpeed'] = edges['maxSpeed'].apply(convertMaxSpeedToInt)
    
    # Add new noWay column
    edges['noWay'] = 0
    
    edges['weight'] = float('inf')
    
    edges['speedOrMaxSpeed'] = int(1)
    
    add_full_traffic_data(edges)
    
    fixWrongDataE(edges)
    
    return ox.graph_from_gdfs(nodes, edges)

In [ ]:
nodes, edges = ox.graph_to_gdfs(graph)

edges['speedOrMaxSpeed'] = 1

g2 = ox.graph_from_gdfs(nodes, edges)
g2.edges(data=True)
#graph.edges(data=True)

In [27]:
type(edges['speedOrMaxSpeed'].iloc[0])

numpy.int64

In [25]:
ox.save_graphml(g2)

In [ ]:
import random

In [ ]:
#graph.edges(data=True)
dummyG = graph.copy()

In [ ]:
#nodes, edges = ox.graph_to_gdfs(G)

for (i, j, data) in dummyG.edges(data=True):
    dummyG[i][j][0]['weight'] = random.randint(1, 20)


In [ ]:
dummyG.edges(data=True)

In [ ]:
random.choice(list(dummyG.nodes()))
# 25632859
# 1685267212

In [ ]:
nx.shortest_path(dummyG, source=25632859, target=1685267212, weight="weight")